# Imports utilized

In [ ]:
import matplotlib.pyplot as plt
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
 

import pandas as pd
import geopandas as gpd
import maup
from maup.smart_repair import smart_repair
import time

# Vest Shape File

In [ ]:
precincts2020 = gpd.read_file('./in_vest_20/in_vest_20.shp')